In [9]:
%pip install catboost
%pip install lightgbm
%pip install xgboost
%pip install imblearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 13.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 7.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 4.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 14.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Data encoding and split

In [1]:
import pandas as pd
import pickle

df = pd.read_pickle("imputated_clustered_data.pkl")
df.head(10)

,ACCT_NBR,SUCCESSOR,MS_METER_NBR,BS_RATE,time_series,label,XRHSH,VOLTAGE,PARNO,CONTRACT_CAPACITY,ACCT_CONTROL,ACCT_WGS84_X,ACCT_WGS84_Y,SUPPLIER,SUPPLIER_TO,REQUEST_TYPE,COMPL_REQUEST_STATUS,cluster_labels
0,000A31FE38C6F648845941B642E287FE0200000000004A...,1,000A31FE38C6F648845941B642E287FE020000007FDE53...,21,"[0, 0, 0, 0, 0, 0.23803191489361705, 0.2327127...",0,2.0,LOW,0.0,12.0,0.0,"53,422551543686701","19,592757873078753",Δ,Δ,,,-1
1,000A31FE38C6F648845941B642E287FE0200000000004A...,2,000A31FE38C6F648845941B642E287FE0200000040D2A7...,21,"[0, 0, 0, 0, 0, 0.23803191489361705, 0.2327127...",0,2.0,LOW,0.0,12.0,0.0,"53,422551543686701","19,592757873078753",Δ,,,,-1
2,000A31FE38C6F648845941B642E287FE0200000000004A...,2,000A31FE38C6F648845941B642E287FE020000007FDE53...,21,"[0, 0, 0, 0, 0, 0.23803191489361705, 0.2327127...",0,2.0,LOW,0.0,12.0,0.0,"53,422551543686701","19,592757873078753",Δ,,,,-1
3,000A31FE38C6F648845941B642E287FE02000000000050...,1,000A31FE38C6F648845941B642E287FE0200000099C3B6...,10,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1.0,LOW,0.0,12.0,0.0,"53,424082335108444","19,570173536681853",Δ,,,,65
4,000A31FE38C6F648845941B642E287FE02000000000050...,1,000A31FE38C6F648845941B642E287FE0200000099C3B6...,12,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1.0,LOW,0.0,12.0,0.0,"53,424082335108444","19,570173536681853",Δ,,,,65
5,000A31FE38C6F648845941B642E287FE0200000000005D...,1,000A31FE38C6F648845941B642E287FE0200000044E98C...,21,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,8.0,LOW,0.0,12.0,0.0,"53,47015026955372","19,592456022809351",Δ,,,,317
6,000A31FE38C6F648845941B642E287FE0200000000007D...,1,000A31FE38C6F648845941B642E287FE020000003A5267...,10,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1.0,LOW,0.0,12.0,0.0,"53,440751323514696","19,598127268868772",Δ,Ω,reprChange,compl,367
7,000A31FE38C6F648845941B642E287FE0200000000007D...,2,000A31FE38C6F648845941B642E287FE020000003A5267...,10,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1.0,LOW,0.0,12.0,0.0,"53,440751323514696","19,598127268868772",Ω,,,,367
8,000A31FE38C6F648845941B642E287FE0200000000008D...,5,000A31FE38C6F648845941B642E287FE0200000096ED89...,21,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,2.0,LOW,2.0,25.0,0.0,"53,407108521588633","19,569743639932696",Δ,,,compl,110
9,000A31FE38C6F648845941B642E287FE0200000000008D...,5,000A31FE38C6F648845941B642E287FE0200000096ED89...,21,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,2.0,LOW,2.0,25.0,0.0,"53,407108521588633","19,569743639932696",Δ,,,,110


In [2]:
# Find the maximum length of time_series
max_length = df['time_series'].apply(len).max()

# Create new columns and fill them with the values from time_series
for i in range(max_length):
    column_name = f'measurement_{i}'
    df[column_name] = df['time_series'].apply(lambda x: x[i] if i < len(x) else None)

df.drop(['ACCT_NBR', 'SUCCESSOR', 'MS_METER_NBR', 'ACCT_WGS84_X', 'ACCT_WGS84_Y', 'time_series'], axis=1, inplace=True)

df.head(10)

,BS_RATE,label,XRHSH,VOLTAGE,PARNO,CONTRACT_CAPACITY,ACCT_CONTROL,SUPPLIER,SUPPLIER_TO,REQUEST_TYPE,...,measurement_50,measurement_51,measurement_52,measurement_53,measurement_54,measurement_55,measurement_56,measurement_57,measurement_58,measurement_59
0,21,0,2.0,LOW,0.0,12.0,0.0,Δ,Δ,,...,0.542121,0.526889,0.563361,0.676491,0.541044,0.545747,0.088889,0.244444,0.570370,1.000000
1,21,0,2.0,LOW,0.0,12.0,0.0,Δ,,,...,0.542121,0.526889,0.563361,0.676491,0.944882,0.267717,0.275591,0.322835,1.000000,0.338583
2,21,0,2.0,LOW,0.0,12.0,0.0,Δ,,,...,0.542121,0.526889,0.563361,0.676491,0.541044,0.545747,0.495319,0.509675,0.776119,1.000000
3,10,0,1.0,LOW,0.0,12.0,0.0,Δ,,,...,0.292548,0.519688,0.463716,0.328018,1.000000,0.325415,0.472177,0.403514,0.993492,0.393752
4,12,0,1.0,LOW,0.0,12.0,0.0,Δ,,,...,0.236234,0.512433,0.396092,0.264654,1.000000,0.259325,0.495560,0.353464,0.957371,0.329485
5,21,0,8.0,LOW,0.0,12.0,0.0,Δ,,,...,0.000000,0.000000,0.777458,1.000000,0.736046,0.170688,0.280879,0.151963,0.150882,0.190853
6,10,0,1.0,LOW,0.0,12.0,0.0,Δ,Ω,reprChange,...,0.938664,0.783133,0.732202,1.000000,0.994524,0.605696,0.934283,0.648412,0.483023,0.123220
7,10,0,1.0,LOW,0.0,12.0,0.0,Ω,,,...,0.302083,0.318452,0.208707,0.574302,0.777346,0.515901,0.996466,0.977032,1.000000,0.902827
8,21,0,2.0,LOW,2.0,25.0,0.0,Δ,,,...,0.750824,0.641892,0.637937,0.632993,0.832070,0.581905,0.483191,1.000000,0.403098,0.443968
9,21,0,2.0,LOW,2.0,25.0,0.0,Δ,,,...,0.750824,0.641892,0.637937,0.632993,0.832070,0.581905,0.483191,1.000000,0.403098,0.443968


In [3]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

# Apply pd.get_dummies on the initial dataset
one_hot_encoded_data = pd.get_dummies(df, columns=categorical_columns)

# Save the column names
one_hot_columns = one_hot_encoded_data.columns

# Save the one_hot_columns to a pickle file
with open("Models/one_hot_columns.pkl", "wb") as f:
    pickle.dump(one_hot_columns, f)

X = one_hot_encoded_data.drop(["label"], axis=1)
y = one_hot_encoded_data["label"]

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=80, stratify=y)
X_detectors_train, X_agent_train, y_detectors_train, y_agent_train = train_test_split(X_train, y_train, test_size=0.5, random_state=80, stratify=y_train)

# Detectors processing

In [10]:
import pandas as pd
from Agent.Sensors.Catboost import CatBoostAnomalyDetector
from Agent.Sensors.LightGBM import LightGBMAnomalyDetector
from Agent.Sensors.XGBoost import XGBoostAnomalyDetector
from Agent.Sensors.RandomForest import RandomForestAnomalyDetector
import numpy as np
import pickle

In [11]:
#  (Number of outliers / Total number of data points)
outlier_fraction = (len(y_detectors_train[y_detectors_train == 1]) / X_detectors_train.shape[0])

# Class importance weights
weight_minority = 0.7
weight_majority = 0.3

# Calculate class weights for each sample
sample_weights = np.array([weight_minority if y == 1 else weight_majority for y in y_detectors_train])

# Instantiate the detectors
detectors = {
    "RandomForest": RandomForestAnomalyDetector(max_depth=25, random_state=0, outlier_fraction=outlier_fraction, class_weight={0: weight_majority, 1: weight_minority}, sample_weights=sample_weights),
    "XGBoost": XGBoostAnomalyDetector(outlier_fraction=outlier_fraction, class_weight=sample_weights, sample_weights=sample_weights),
    "CatBoost": CatBoostAnomalyDetector(outlier_fraction=outlier_fraction, class_weight=[weight_majority, weight_minority], sample_weights=sample_weights),
    "LightGBM": LightGBMAnomalyDetector(outlier_fraction=outlier_fraction, class_weight={0: weight_majority, 1: weight_minority}, sample_weights=sample_weights),
}

# Train the detectors and store their predict_proba results in a new DataFrame called sensors_df
sensors_df = pd.DataFrame()
for name, detector in detectors.items():
    print("Current detector: {}".format(name))
    detector.fit(X_detectors_train, y_detectors_train)
    detector.evaluate(X_agent_train, y_agent_train)
    sensors_df[name] = detector.proba


Current detector: RandomForest
Accuracy (Random Forest): 0.9922
Precision (Random Forest): 0.5561
Recall (Random Forest): 0.5564
F1 score (Random Forest): 0.5563
F2 score (Random Forest): 0.5564
Current detector: XGBoost
Accuracy (XGBoost): 0.9887
Precision (XGBoost): 0.3565
Recall (XGBoost): 0.3567
F1 score (XGBoost): 0.3566
F2 score (XGBoost): 0.3567
Current detector: CatBoost
Accuracy (CatBoost): 0.9912
Precision (CatBoost): 0.5004
Recall (CatBoost): 0.5007
F1 score (CatBoost): 0.5006
F2 score (CatBoost): 0.5006
Current detector: LightGBM
Accuracy (LGBM): 0.9884
Precision (LGBM): 0.3410
Recall (LGBM): 0.3410
F1 score (LGBM): 0.3410
F2 score (LGBM): 0.3410


In [12]:
sensors_df.to_pickle("sensors_df.pkl")
models_to_save = detectors.copy()
# Save all the models in one pickle file
with open("Models/detectors.pkl", "wb") as f:
    pickle.dump(models_to_save, f)

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score
import numpy as np

# Class importance weights
weight_minority = 0.7
weight_majority = 0.3

# Calculate class weights for each sample
test_sample_weights = np.array([weight_minority if y == 1 else weight_majority for y in y_test])

for name, detector in detectors.items():
    detector.sample_weights = test_sample_weights
    detector.evaluate(X_test, y_test)
    print("")

Accuracy (Random Forest): 0.9872
Precision (Random Forest): 0.7465
Recall (Random Forest): 0.5580
F1 score (Random Forest): 0.6386
F2 score (Random Forest): 0.5876

Accuracy (XGBoost): 0.9813
Precision (XGBoost): 0.5580
Recall (XGBoost): 0.3511
F1 score (XGBoost): 0.4310
F2 score (XGBoost): 0.3792

Accuracy (CatBoost): 0.9857
Precision (CatBoost): 0.7027
Recall (CatBoost): 0.5032
F1 score (CatBoost): 0.5864
F2 score (CatBoost): 0.5335

Accuracy (LGBM): 0.9810
Precision (LGBM): 0.5491
Recall (LGBM): 0.3429
F1 score (LGBM): 0.4222
F2 score (LGBM): 0.3707

